## DataSet 
因为利用 torchvision.datasets 的数据集时，下载的太慢了，所以想要直接载入自己本地的 CIFAR10 数据集。于是就学习了一下。

Pytorch 中加载数据首先是弄数据集 DataSet,然后利用 DataLoader 进行数据的载入。

我们要继承 from torch.utils.data import Dataset 的Dataset类，然后主要是重写 __init__，__getitem__和_len方法。可以仿照官网给的 CIFAR10 数据集的源码来仿写。注意在 getiem 中实现对数据集的预处理，也就是传进来的 torchvision.transforms 还有 __getitem__() 方法同时返回 data和target,就是元组，数据集和数据的载入在 pytorch 中都是用了元组保存。

## DataLoader
在定义数据集之后，就是要用 DataLoader 进行数据的载入了。在 DataLoader 中主要注意 batch_size，sampler 和 collate_fn。
batch_size 就会把原本的数据分成一个个小 batch_size 再返回。sampler 决定了如何取数据。在官网中的 DataLoader 里有这么一段

```python
if batch_sampler is None:
            if sampler is None:
                if shuffle:
                    sampler = RandomSampler(dataset)
                else:
                    sampler = SequentialSampler(dataset)
            batch_sampler = BatchSampler(sampler, batch_size, drop_last)
        self.sampler = sampler
        self.batch_sampler = batch_sampler
        self.__initialized = True
```
在 DataLoader 中 shuffle 是指定在载入数据时，是否混乱载入，我们可以看到，混乱载入就是用了一个 RandomSampler 在加载我们的数据集。
而不混乱载入就是利用 SequentialSampler 按顺序的载入数据。Sampler 里面是怎么实现的等下再看。注意这里有 sampler 和 batch_sampler，首先就是先用 sampler 取数据，在利用 batchSampler 按 batch_size 大小再分。 在这里注意 sampler 和 batch_sampler 返回的数据都是迭代器类型。

然后我们看看这些 Sampler.
``` 
    class SequentialSampler(Sampler):
    def __init__(self, data_source):
        self.data_source = data_source

    def __iter__(self):
        return iter(range(len(self.data_source)))

[docs]class RandomSampler(Sampler):
    def __init__(self, data_source, replacement=False, num_samples=None):
        self.data_source = data_source
        self.replacement = replacement
        self._num_samples = num_samples

    def __iter__(self):
        n = len(self.data_source)
        if self.replacement:
            return iter(torch.randint(high=n, size=(self.num_samples,), dtype=torch.int64).tolist())
        return iter(torch.randperm(n).tolist())
```
删除了一下代码。我们可以发现 Sampler 就是返回一个 index 列表迭代器，然后就可以根据索引取数据，就达到了如何取数据的效果。

再看一下 BatchSampler:
```
[docs]class BatchSampler(Sampler):
    Example:
        >>> list(BatchSampler(SequentialSampler(range(10)), batch_size=3, drop_last=False))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]
        >>> list(BatchSampler(SequentialSampler(range(10)), batch_size=3, drop_last=True))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    """
    def __init__(self, sampler, batch_size, drop_last):

        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last

    def __iter__(self):
        batch = []
        for idx in self.sampler:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yield batch
                batch = []
        if len(batch) > 0 and not self.drop_last:
            yield batch
```
官网还再上面给了一个例子。我们可以看到在 __iter__ 中，batchSamper 是用上一个 sampler 返回的 index 迭代器，然后根据 batch_size 把索引再组合一遍，返回的就是一个个块，块里就是索引值。

## collate_fn
就是在取数据的时候如果我要数据进行一些其他的操作，可以自定义处理过程。我们看官网给的一个例子：
``` 
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)
inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
dataset = TensorDataset(inps, tgts)

loader = DataLoader(dataset, batch_size=2, collate_fn=collate_wrapper,
                    pin_memory=True)

for batch_ndx, sample in enumerate(loader):
    print(sample.inp.is_pinned())
    print(sample.tgt.is_pinned())
```
在这里想提一点，在 SimpleCustomBatch 中拿到的数据是 [($x_{1},y_{1}$),($x_{2},y_{2}$)] 这种样子的，而 DataLoader 返回的数据是
([$x_{1},x_{2}...$],[$y_{1},y_{2}..$]) 这样子的，如果再有 batch_size 的话，就是([batch_index],[$x_{1},x_{2}...$],[$y_{1},y_{2}..$])。所以在 SimpleCustomBatch 中有一个解包，并且返回的都是解包后的元组。